In [16]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

In [26]:
# Twitter credentials
# Obtain them from your twitter developer account
consumer_key = "Cmzu3Ae652CMqg1GjLp9jItln"
consumer_secret = "zKjchoElqhGPGoeO0CT5fDexzlNMp2574jUrVl1j8rPKjDRHJI"
access_key = "1301496986017783808-cZ0AFGnj7lfksV7TWRgIwcPd7vtORO"
access_secret = "cVqXqNK7bhH3dK75cnff0TAEeMnxAQKomwnq0f7KjPhLa"

# Pass your twitter credentials to tweepy via its OAuthHandler
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [27]:
lol = None
def scraptweets(search_words, date_since, numTweets, numRuns):
    global lol
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['tweet_date','poi_id','poi_name','tweet_text','country','tweet_lang','hashtags','verified','tweet_urls','mentions','retweetcount'])
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()


       
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        #tweets = tweepy.Cursor(api.search, q=(search_words), lang="hi", since=date_since, tweet_mode='extended').items(numTweets)
        tweets = tweepy.Cursor(api.user_timeline, screen_name='@AmitShah',since=date_since, tweet_mode="extended").items(numTweets)
        # Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
        for tweet in tweet_list:
    
    
            #if api.verify_credentials() == False: 
            #    ver='false' 
            #else: 
             #   ver='true' 
# Pull the values
            tweet_date = tweet.created_at
            poi_id = tweet.user.id
            poi_name = tweet.user.screen_name
            try:
                tweet_text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                tweet_text = tweet.full_text
            country = "India"
            tweet_lang = tweet.lang
            hashtags = tweet.entities['hashtags']
            verified = tweet.user.verified
            tweet_urls = tweet.entities['urls']
            mentions = tweet.entities['user_mentions']
            retweetcount = tweet.retweet_count
            
# Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [tweet_date,poi_id,poi_name,tweet_text,country,tweet_lang,hashtags,verified,tweet_urls,mentions,retweetcount]
# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
    #    time.s+leep(920) #15 minute sleep time
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
   # to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    #path = os.getcwd()
# Store dataframe in csv with creation date timestamp
#    db_tweets.to_csv(filename, index = False)
    db_tweets.tweet_date = db_tweets.tweet_date.astype(str)
    lol = db_tweets.to_dict(orient='records')
   # db_tweets.to_json(r'/Users/manasachalla/Desktop/IRProject/data1.json')
   
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrape is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [28]:
search_words = "#covid19"
date_since = "2020-07-07"
numTweets = 3000
numRuns = 1
scraptweets(search_words, date_since, numTweets, numRuns)


no. of tweets scraped for run 1 is 3000
time take for 1 run to complete is 2.01 mins
Scraping has completed!
Total time taken to scrape is 2.0166666666666666 minutes.


In [29]:
lol[0]

{'tweet_date': '2020-09-20 13:16:03',
 'poi_id': 1447949844,
 'poi_name': 'AmitShah',
 'tweet_text': 'मोदी सरकार के यह कृषि सुधार हमारे किसान भाइयों का हक मारने वाले बिचौलियों से उन्हें मुक्त करेंगे और साथ ही उनकी उपज को कहीं भी बेचने व उसका सही दाम दिलवाकर उनकी आय बढ़ाने में एक सकारात्मक भूमिका निभाएँगे।\n\nइस निर्णय के बाद भी MSP की व्यवस्था बनी रहेगी व सरकारी खरीद भी जारी रहेगी।',
 'country': 'India',
 'tweet_lang': 'hi',
 'hashtags': [],
 'verified': True,
 'tweet_urls': [],
 'mentions': [],
 'retweetcount': 3230}

In [30]:
with open('AmitShah.json', 'w') as fout:
    json.dump(lol , fout)